In [ ]:
import streamlit as st
import pandas as pd
import os
import geopandas as gpd

# Caminho para o arquivo shapefile
shapefile_path = os.path.join(os.getcwd(), 'shapefile', 'PB_Municipios_2022.shp')  # Certifique-se de que o nome do arquivo está correto

# Carregar o shapefile
data = gpd.read_file(shapefile_path)

# Salvar como GeoJSON
data.to_file('municipios_paraiba.geojson', driver='GeoJSON')

# Caminho relativo para o diretório onde estão os arquivos .csv
diretorio_csv = os.path.join(os.getcwd(), 'Dados_csv')

# Função para carregar dados diretamente dos arquivos CSV
def load_data(diretorio_csv):
    dataframes = []
    for arquivo in os.listdir(diretorio_csv):
        if arquivo.endswith('.csv'):
            caminho_completo = os.path.join(diretorio_csv, arquivo)
            try:
                # Definir os tipos de dados esperados para as colunas problemáticas
                dtypes = {
                    'IDADEMAE': 'float64',  # Exemplo, ajuste conforme necessário
                    'CODMUNNASC': 'str',     # Manter como string
                    'PESO': 'str'            # Manter como string antes da conversão
                }
                
                # Carregar cada arquivo CSV em um DataFrame e adicionar à lista
                df = pd.read_csv(caminho_completo, dtype=dtypes)
                dataframes.append(df)
            except Exception as e:
                st.error(f'Erro ao ler o arquivo {arquivo}: {e}')
    
    # Concatenar todos os DataFrames em um único DataFrame, se houver dados
    if dataframes:
        df_total = pd.concat(dataframes, ignore_index=True)
        return df_total
    else:
        st.warning('Nenhum arquivo .csv encontrado ou lido.')
        return None

# Carregar os dados CSV
df_total = load_data(diretorio_csv)

# Função para pré-processamento dos dados
def preprocess_data(df):

    try:
        # Converte a coluna DTNASC para datetime
        df['DTNASC'] = pd.to_datetime(df['DTNASC'], format='%d%m%Y', errors='coerce')
        
        # Converte a coluna PESO para string, substitui vírgulas por pontos e remove espaços
        df['PESO'] = df['PESO'].astype(str).str.replace(',', '.').str.strip()
        
        # Converter para float, forçando valores inválidos a serem NaN
        df['PESO'] = pd.to_numeric(df['PESO'], errors='coerce')
        
        # Converter a idade da mãe para numérico
        df['IDADEMAE'] = pd.to_numeric(df['IDADEMAE'], errors='coerce')

        # Garantir que CODMUNNASC seja uma string
        df['CODMUNNASC'] = df['CODMUNNASC'].fillna('').astype(str)

        # Filtragem para pesos abaixo de 2500 e códigos de município que começam com '25'
        df_baixo_peso = df[(df['PESO'] < 2500) & (df['PESO'] > 0) & (df['CODMUNNASC'].str.startswith('25'))].reset_index(drop=True)
        
        # Adiciona a coluna 'Ano' ao DataFrame df_baixo_peso
        df_baixo_peso['Ano'] = df_baixo_peso['DTNASC'].dt.year

        return df_baixo_peso
    except Exception as e:
        st.error(f"Erro durante o pré-processamento: {e}")
        return None


# Inicializa df_baixo_peso como None
df_baixo_peso = None

# Verifique as colunas do GeoDataFrame
if 'CD_MUN' in data.columns and 'NM_MUN' in data.columns:
    municipios_mapping = data[['CD_MUN', 'NM_MUN']].copy()
    municipios_mapping.columns = ['CODMUNNASC', 'Nome_Municipio']
else:
    raise ValueError("As colunas 'CD_MUN' e 'NM_MUN' não estão presentes no GeoDataFrame.")

# Garantir que CODMUNNASC seja string para corresponder com os DataFrames
municipios_mapping['CODMUNNASC'] = municipios_mapping['CODMUNNASC'].astype(str)

# Mesclar o nome do município em df_baixo_peso e df_total
if df_total is not None:
    df_total = df_total.merge(municipios_mapping, on='CODMUNNASC', how='left')
    df_baixo_peso = preprocess_data(df_total)  # Chama a função de pré-processamento
    if df_baixo_peso is not None:
        df_baixo_peso = df_baixo_peso.merge(municipios_mapping, on='CODMUNNASC', how='left')


C:\Users\PICHAU\AppData\Local\Temp\ipykernel_1560\3851348874.py:33: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_completo, dtype=dtypes)
C:\Users\PICHAU\AppData\Local\Temp\ipykernel_1560\3851348874.py:33: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_completo, dtype=dtypes)
C:\Users\PICHAU\AppData\Local\Temp\ipykernel_1560\3851348874.py:33: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_completo, dtype=dtypes)


In [7]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1412699 entries, 0 to 1412698
Data columns (total 73 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   contador        1079936 non-null  float64       
 1   LOCNASC         1412699 non-null  int64         
 2   CODMUNNASC      1412699 non-null  object        
 3   IDADEMAE        1410482 non-null  float64       
 4   ESTCIVMAE       1392277 non-null  float64       
 5   ESCMAE          1380622 non-null  float64       
 6   QTDFILVIVO      1336925 non-null  float64       
 7   QTDFILMORT      1273027 non-null  float64       
 8   CODMUNRES       1412699 non-null  int64         
 9   GESTACAO        1396306 non-null  float64       
 10  GRAVIDEZ        1410485 non-null  float64       
 11  PARTO           1410278 non-null  float64       
 12  CONSULTAS       1408449 non-null  float64       
 13  DTNASC          1405476 non-null  datetime64[ns]
 14  SEXO            14

In [11]:
df_baixo_peso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98055 entries, 0 to 98054
Data columns (total 75 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   contador          72346 non-null  float64       
 1   LOCNASC           98055 non-null  int64         
 2   CODMUNNASC        98055 non-null  object        
 3   IDADEMAE          97956 non-null  float64       
 4   ESTCIVMAE         96951 non-null  float64       
 5   ESCMAE            96292 non-null  float64       
 6   QTDFILVIVO        94019 non-null  float64       
 7   QTDFILMORT        91752 non-null  float64       
 8   CODMUNRES         98055 non-null  int64         
 9   GESTACAO          96758 non-null  float64       
 10  GRAVIDEZ          97937 non-null  float64       
 11  PARTO             97952 non-null  float64       
 12  CONSULTAS         97784 non-null  float64       
 13  DTNASC            97558 non-null  datetime64[ns]
 14  SEXO              9805

In [ ]:
df_total.to_parquet('df_total.parquet')
df_baixo_peso.to_parquet('df_baixo_peso.parquet')